# Managing TwelveLabs Indexes

This notebook demonstrates how to manage video indexes using the TwelveLabs API, including creating, listing, retrieving, updating, and deleting indexes.

## Documentation
- Official API Reference: https://docs.twelvelabs.io/v1.3/sdk-reference/python/manage-indexes

## Overview
Indexes are containers that organize your video content. This notebook covers all CRUD (Create, Read, Update, Delete) operations for managing indexes.

### # The index object

The `index` object is composed of the following fields:

* `_id`: A string representing the unique identifier of the index. It is assigned by the platform when an index is created.
* `index_name`: A string representing the name of the index.
* `models`: An array that specifies the [video understanding models](/v1.3/docs/concepts/models) and the [model options](/v1.3/docs/concepts/modalities#model-options) that are enabled for this index. Models determine what tasks you can perform with your videos. Model options determine which modalities the platform analyzes.
* `created_at`: A string representing the date and time, in the RFC 3339 format, that the index was created.
* `updated_at`: A string representing the date and time, in the RFC 3339 format, that the index was updated.
* `expires_at`: A string representing the date and time, in the RFC 3339 format, when your index will expire.
* `video_count`: An integer representing the number of videos in the index.
* `total_duration`: An integer representing the total duration of the videos in the index.
* `addons`: The list of add-ons that are enabled for this index.


In [ ]:
# Import required libraries for TwelveLabs API interaction
import uuid  # For generating unique identifiers if needed
from twelvelabs import TwelveLabs  # Main TwelveLabs client class
from twelvelabs.indexes import IndexesCreateRequestModelsItem  # Type hint for index model configuration
from twelvelabs.tasks import TasksRetrieveResponse  # Type hint for task response objects

d:\projects\12labs\new1\.venv\Lib\site-packages\twelvelabs\core\pydantic_utilities.py:13: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.datetime_parse import parse_date as parse_date


In [ ]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv  

# Load environment variables (including API key)
# IMPORTANT: Make sure to actually call load_dotenv() to load variables from .env file
load_dotenv()

# Retrieve the TwelveLabs API key from environment variables
apiKey = os.getenv("TL_API_KEY")

# Verify that the API key was successfully loaded
# This prints True if key exists, False otherwise
print(f"Key loaded: {bool(apiKey)}")

Key loaded: True


## Create an Index

Create a new index with specified models and configuration. The index will be used to organize and process your video content.


In [ ]:
# Import libraries for making HTTP requests and handling JSON data
import requests  # For making HTTP POST requests to the API
import json  # For formatting JSON output

# TwelveLabs API endpoint for creating indexes
url = "https://api.twelvelabs.io/v1.3/indexes"

# Configuration payload for the new index
# This defines what models and features will be used for video processing
payload = {
    "index_name": "myIndex",  # Name of the index to create
    "models": [
        {
            "model_name": "marengo3.0",  # Advanced video understanding model
            "model_options": ["visual", "audio"]  # Process both visual and audio content
        },
        {
            "model_name": "pegasus1.2",  # Video analysis and summarization model
            "model_options": ["visual", "audio"]  # Process both visual and audio content
        }
    ],
    "addons": ["thumbnail"]  # Enable thumbnail generation for indexed videos
}

# HTTP headers required for API authentication
headers = {
    "x-api-key": apiKey,  # Your TwelveLabs API key
    "Content-Type": "application/json"  # Specify JSON content type
}

# Send POST request to create the index
response = requests.post(url, json=payload, headers=headers)

# Print the API response in a formatted JSON structure
# The response includes the index ID (_id) which you'll need for future operations
print(json.dumps(response.json(), indent=2))

{'_id': '6953934ddb0246c06ce304d7'}


## List Indexes

Retrieve a list of all indexes, optionally filtered by various criteria such as index name, model options, or creation date.


In [ ]:
# Import libraries for making HTTP requests and handling JSON data
import requests  # For making HTTP GET requests to the API
import json  # For formatting JSON output

# TwelveLabs API endpoint for listing indexes
url = "https://api.twelvelabs.io/v1.3/indexes"

# Query parameters for filtering the list of indexes
# You can filter by various criteria to narrow down results
querystring = {
    "index_name": "",  # Filter by index name (empty string = no filter)
    "model_options": "visual,audio",  # Filter by model options (visual and/or audio)
    "model_family": "marengo",  # Filter by model family (e.g., "marengo", "pegasus")
    # Uncomment the following lines to filter by date range:
    #"created_at": "2024-08-16T16:53:59Z",  # Filter by creation date (RFC 3339 format)
    #"updated_at": "2024-08-16T16:55:59Z"  # Filter by last update date (RFC 3339 format)
}

# HTTP headers required for API authentication
headers = {"x-api-key": apiKey}

# Send GET request to retrieve the list of indexes
response = requests.get(url, headers=headers, params=querystring)

# Print the API response in a formatted JSON structure
# The response includes an array of index objects and pagination information
print(json.dumps(response.json(), indent=2))

{'data': [{'_id': '6953934ddb0246c06ce304d7', 'index_name': 'myIndex', 'created_at': '2025-12-30T08:54:37.75Z', 'updated_at': '2025-12-30T08:54:37.75Z', 'expires_at': '2026-03-30T08:54:37.75Z', 'addons': ['thumbnail'], 'models': [{'model_name': 'marengo3.0', 'model_options': ['visual', 'audio'], 'finetuned': False}, {'model_name': 'pegasus1.2', 'model_options': ['visual', 'audio'], 'finetuned': False}], 'video_count': 0, 'total_duration': 0}, {'_id': '69538a69db0246c06ce302ac', 'index_name': 'MY cool index', 'created_at': '2025-12-30T08:16:41.789Z', 'updated_at': '2025-12-30T08:46:39.942Z', 'expires_at': '2026-03-30T08:16:41.789Z', 'addons': ['thumbnail'], 'models': [{'model_name': 'marengo3.0', 'model_options': ['visual', 'audio'], 'finetuned': False}, {'model_name': 'pegasus1.2', 'model_options': ['visual', 'audio'], 'finetuned': False}], 'video_count': 0, 'total_duration': 0}], 'page_info': {'page': 1, 'limit_per_page': 10, 'total_page': 1, 'total_results': 2}}


## Retrieve an Index

Get detailed information about a specific index by its ID, including configuration, statistics, and metadata.


In [ ]:
# Import libraries for making HTTP requests and handling JSON data
import requests  # For making HTTP GET requests to the API
import json  # For formatting JSON output

# Specify the index ID you want to retrieve
# Replace this with your actual index ID
index_id = "69538a69db0246c06ce302ac"

# Construct the API endpoint URL for retrieving a specific index
url = "https://api.twelvelabs.io/v1.3/indexes/" + index_id

# HTTP headers required for API authentication
headers = {"x-api-key": apiKey}

# Send GET request to retrieve the index details
response = requests.get(url, headers=headers)

# Print the API response in a formatted JSON structure
# The response includes all index properties: _id, index_name, models, video_count, etc.
print(json.dumps(response.json(), indent=2))

{'_id': '69538a69db0246c06ce302ac', 'index_name': 'MY cool index', 'created_at': '2025-12-30T08:16:41.789Z', 'updated_at': '2025-12-30T08:46:39.942Z', 'expires_at': '2026-03-30T08:16:41.789Z', 'addons': ['thumbnail'], 'models': [{'model_name': 'marengo3.0', 'model_options': ['visual', 'audio'], 'finetuned': False}, {'model_name': 'pegasus1.2', 'model_options': ['visual', 'audio'], 'finetuned': False}], 'video_count': 0, 'total_duration': 0}


## Update an Index

Modify properties of an existing index, such as changing its name or configuration. Note: Some properties may not be updatable after creation.


In [ ]:
# Import libraries for making HTTP requests and handling JSON data
import requests  # For making HTTP PUT requests to the API
import json  # For formatting JSON output

# Construct the API endpoint URL for updating a specific index
# Replace the index ID with your actual index ID
url = "https://api.twelvelabs.io/v1.3/indexes/69538a69db0246c06ce302ac"

# HTTP headers required for API authentication
headers = {
    "x-api-key": apiKey,  # Your TwelveLabs API key
    "Content-Type": "application/json"  # Specify JSON content type
}

# Payload containing the fields to update
# Only include fields that you want to modify
payload = {
    "index_name": "MY cool index"  # Example: Update the index name
    # Add other updatable fields here as needed
}

# Send PUT request to update the index
response = requests.put(url, json=payload, headers=headers)

# Print the HTTP status code
# 204 (No Content) typically indicates successful update
print(f"Status Code: {response.status_code}")

# Try to parse and print JSON response
# Some successful updates return empty responses (status 204)
try:
    print(json.dumps(response.json(), indent=2))
except Exception:
    # If response is not JSON (e.g., empty body for 204), print raw text
    print(f"Raw text: {response.text}")

Status Code: 204
Response is not JSON (likely empty or HTML).
Raw text: 


## Delete an Index

Permanently delete an index and all its associated data. **Warning: This action cannot be undone!** Make sure you have the correct index ID before proceeding.


In [ ]:
# Import library for making HTTP requests
import requests  # For making HTTP DELETE requests to the API

# IMPORTANT: Double check this ID is the one you want to delete!
# Deleting an index is permanent and cannot be undone
index_id = "6952ac44e5676b2af30d5330"

# Construct the API endpoint URL for deleting a specific index
url = f"https://api.twelvelabs.io/v1.3/indexes/{index_id}"

# HTTP headers required for API authentication
headers = {"x-api-key": apiKey}

# Send DELETE request to remove the index
response = requests.delete(url, headers=headers)

# Check the response status code
# 204 (No Content) or 200 (OK) typically indicates successful deletion
if response.status_code == 204:
    print("Success! Index deleted.")
else:
    # If deletion failed, print the error status and message
    print(f"Index delete failed: {response.status_code}")
    print(response.text)

Success! Index deleted.
